In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib as mpl

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

sys.path.append("../python/")

from plotutils import *
from metricutils import *

In [ ]:
project_root  = ".."
base_folders_raw = ["SIR_simulation_chain", "SIR_simulation_ring", "SIR_simulation_star"]

nodes = ["H", "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8"]
nodes_dict = {"H":"M1", "L1":"M2", "L2":"M3", "L3":"M4", "L4":"M5", "L5":"M6", "L6":"M7", "L7":"M8", "L8":"M9"}

In [ ]:
R0 = "2"
topology = "star"
hub = "M2"
target_folder = f"../outputs/SIR_simulation_{topology}/"

best_delta = 5 #bests[(R0,topology)][0]
best_omega = 8 #bests[(R0,topology)][1]

te_folder = os.path.join(target_folder, f"run_w{best_omega}_d{best_delta}_R_risk_ij/")
te_folder_cases = os.path.join(target_folder, f"run_w{best_omega}_d{best_delta}_R_new_cases_by_100k/")

Txy = xr.load_dataarray(os.path.join(te_folder, "TE.nc"))
DIxy = calculate_TSxy(Txy)

Txy_cases = xr.load_dataarray(os.path.join(te_folder_cases, "TE.nc"))
DIxy_cases = calculate_TSxy(Txy_cases)

params_fname = os.path.join(te_folder,"params.json")
with open(params_fname) as fh:
    params = json.load(fh)
omega = params["te_params"]["omega"]
delta = params["te_params"]["delta"]

cases_ds = xr.open_dataset(os.path.join(target_folder, "cases_pop_ds.nc"))
mobility_matrix = np.load(f"../data/SIR_mobility_matrices/mobility_{topology}_9.npy")

params_fname = os.path.join(target_folder, "config.json")
with open(params_fname) as fh:
    params = json.load(fh)
r0 = params["model_params"]["r0"]
mu = params["model_params"]["mu"]

fig,axs = plt.subplots(1, 2, figsize=(12,4) ,dpi=300)
fig.set_facecolor('white')
for ax in axs.flatten():
    ax.set_anchor('W')

plot_pop_on_graph(mobility_matrix, nodes, nodes_dict, topology, axs[0], hub)
axs[0].set_title("Mobility network")
axs[0].set_aspect(0.9)
axs[0].text(-0.1, 1.02, "A", fontsize=18, 
            transform=axs[0].transAxes, weight='bold', color='#333333')

n_days = 70
interval = 10
date_beg, date_end = plot_incidence(cases_ds, interval, n_days, axs[1], nodes_dict)
axs[1].set_title("Dynamics of the epidemic process ($R_0={:.1f}$, $\mu={:.2f}$)".format(r0,mu))
axs[1].set_aspect(2.85)
axs[1].text(-0.1, 1.02, "B", fontsize=18, 
            transform=axs[1].transAxes, weight='bold', color='#333333')

for ax in axs.flatten():
    ax.set_anchor('NW')
    
fig.savefig("../figures/main/Figure2AB.tiff", dpi=300)

#-----------------------------------------------------------------------------------------------------------------#

patches = nodes
mov = mobility_matrix 
edges_list = []
for i,pi in enumerate(patches):
    for j,pj in enumerate(patches):
        if mov[i,j] != 0 and i != j:
            edges_list.append((nodes_dict[pi],nodes_dict[pj], mov[i,j]))

G = nx.DiGraph()
G.add_nodes_from([nodes_dict[x] for x in patches])

for item in edges_list:
    G.add_edge(item[0],item[1],weight=int(item[2]))

if topology == "star":
    center_node="M1"
    edge_nodes = set(G) - {center_node}
    pos = nx.circular_layout(G.subgraph(edge_nodes))
    pos[center_node] = np.array([0, 0])
else:
    pos = nx.circular_layout(G)
    
node_sizes = 500

DIx_norm = calculate_normalized_TSx(Txy)
DIx_cases_norm = calculate_normalized_TSx(Txy_cases)

domain_min = -1.0
domain_max = 1.0
#domain_min = 0.0
#domain_max = cases_ds["new_cases_by_100k"].max()

norm = mcolors.Normalize(vmin=domain_min, vmax=domain_max)
cmap = sns.color_palette("vlag", as_cmap=True)
#cmap = plt.cm.Purples

all_dates = np.array([np.datetime_as_string(x)[:10] for x in list(DIxy["date"].values)])

days = [12, 15, 18, 21]
days = [10, 15, 25, 35]
dates = all_dates[days]


fig,axs = plt.subplots(2, 4, figsize=(12,6) ,dpi=400, sharex=True, sharey=True,
                       gridspec_kw={"width_ratios":[1,1,1,1.3], "height_ratios":[1,1], "wspace":0.1, "hspace":0.1})


fig.set_facecolor('white')
for ax in axs.flatten():
    ax.set_anchor('W')

    
labels = [f"D{x+delta}" for x in days]
for i, date in enumerate(dates):
    
    if i == 3:
        colorbar = True
    else:
        colorbar = False
    
    nodes_colors = cmap(norm(DIx_norm.loc[:, date]))
    #nodes_colors = cmap(norm(cases_ds["new_cases_by_100k"].loc[:, date]))
    plot_TE_on_graph2(DIxy, nodes, nodes_dict, topology, pos, ax=axs[0,i], hub_idx=hub, date=date,
                      colorbar=colorbar, cbar_type="nodes", nodes_color=nodes_colors)
    
    axs[0,i].set_title(f" ({labels[i]}) "+r"$DI_{XY,risk \rightarrow cases}$")
    axs[0,i].set_axis_off()
    axs[0,i].set_aspect("equal")
    
    nodes_colors = cmap(norm(DIx_cases_norm.loc[:, date]))
    plot_TE_on_graph2(DIxy_cases, nodes, nodes_dict, topology, pos, ax=axs[1,i], hub_idx=hub, date=date,
                      colorbar=colorbar, cbar_type="edges", nodes_color=nodes_colors)
   
    axs[1,i].set_title(f"({labels[i]}) "+r"$DI_{XY,cases \rightarrow cases}$")
    axs[1,i].set_axis_off()
    axs[1,i].set_aspect("equal")

axs[0,0].text(-0.1, 1.02, "C", fontsize=18, 
            transform=axs[0,0].transAxes, weight='bold', color='#333333')
axs[1,0].text(-0.1, 1.02, "D", fontsize=18, 
            transform=axs[1,0].transAxes, weight='bold', color='#333333')

for ax in axs.flatten():
    ax.set_anchor('NW')
    
fig.savefig("../figures/main/Figure2CD.tiff", dpi=300)

In [ ]:
R0 = "2"
topology = "chain"
hub = "M1"
target_folder = f"../outputs/SIR_simulation_{topology}/"

best_delta = 4 #bests[(R0,topology)][0]
best_omega = 9 #bests[(R0,topology)][1]

te_folder = os.path.join(target_folder, f"run_w{best_omega}_d{best_delta}_R_risk_ij/")
te_folder_cases = os.path.join(target_folder, f"run_w{best_omega}_d{best_delta}_R_new_cases_by_100k/")

Txy = xr.load_dataarray(os.path.join(te_folder, "TE.nc"))
DIxy = calculate_TSxy(Txy)

Txy_cases = xr.load_dataarray(os.path.join(te_folder_cases, "TE.nc"))
DIxy_cases = calculate_TSxy(Txy_cases)

params_fname = os.path.join(te_folder,"params.json")
with open(params_fname) as fh:
    params = json.load(fh)
omega = params["te_params"]["omega"]
delta = params["te_params"]["delta"]

cases_ds = xr.open_dataset(os.path.join(target_folder, "cases_pop_ds.nc"))
mobility_matrix = np.load(f"../data/SIR_mobility_matrices/mobility_{topology}_9.npy")

params_fname = os.path.join(target_folder, "config.json")
with open(params_fname) as fh:
    params = json.load(fh)
r0 = params["model_params"]["r0"]
mu = params["model_params"]["mu"]

fig,axs = plt.subplots(1, 2, figsize=(12,4) ,dpi=300)
fig.set_facecolor('white')
for ax in axs.flatten():
    ax.set_anchor('W')

plot_pop_on_graph(mobility_matrix, nodes, nodes_dict, topology, axs[0], hub)
axs[0].set_title("Mobility network")
axs[0].set_aspect(0.9)
axs[0].text(-0.1, 1.02, "A", fontsize=18, 
            transform=axs[0].transAxes, weight='bold', color='#333333')

n_days = 120
interval = 30
date_beg, date_end = plot_incidence(cases_ds, interval, n_days, axs[1], nodes_dict)
axs[1].set_title("Dynamics of the epidemic process ($R_0={:.1f}$, $\mu={:.2f}$)".format(r0,mu))
axs[1].set_aspect(5.25)
axs[1].text(-0.1, 1.02, "B", fontsize=18, 
            transform=axs[1].transAxes, weight='bold', color='#333333')

for ax in axs.flatten():
    ax.set_anchor('NW')

fig.savefig("../figures/supp/FigureS3AB.tiff", dpi=300)

#-----------------------------------------------------------------------------------------------------------------#

patches = nodes
mov = mobility_matrix 
edges_list = []
for i,pi in enumerate(patches):
    for j,pj in enumerate(patches):
        if mov[i,j] != 0 and i != j:
            edges_list.append((nodes_dict[pi],nodes_dict[pj], mov[i,j]))

G = nx.DiGraph()
G.add_nodes_from([nodes_dict[x] for x in patches])

for item in edges_list:
    G.add_edge(item[0],item[1],weight=int(item[2]))

if topology == "star":
    center_node="M1"
    edge_nodes = set(G) - {center_node}
    pos = nx.circular_layout(G.subgraph(edge_nodes))
    pos[center_node] = np.array([0, 0])
else:
    pos = nx.circular_layout(G)
    
node_sizes = 500

DIx_norm = calculate_normalized_TSx(Txy)
DIx_cases_norm = calculate_normalized_TSx(Txy_cases)

domain_min = -1.0
domain_max = 1.0
#domain_min = 0.0
#domain_max = cases_ds["new_cases_by_100k"].max()

norm = mcolors.Normalize(vmin=domain_min, vmax=domain_max)
cmap = sns.color_palette("vlag", as_cmap=True)
#cmap = plt.cm.Purples

all_dates = np.array([np.datetime_as_string(x)[:10] for x in list(DIxy["date"].values)])

days = [12, 15, 18, 21]
days = [11, 16, 26, 36]
dates = all_dates[days]


fig,axs = plt.subplots(2, 4, figsize=(12,6) ,dpi=400, sharex=True, sharey=True,
                       gridspec_kw={"width_ratios":[1,1,1,1.3], "height_ratios":[1,1], "wspace":0.1, "hspace":0.1})


fig.set_facecolor('white')
for ax in axs.flatten():
    ax.set_anchor('W')

    
labels = [f"D{x+delta}" for x in days]
for i, date in enumerate(dates):
    
    if i == 3:
        colorbar = True
    else:
        colorbar = False
    
    nodes_colors = cmap(norm(DIx_norm.loc[:, date]))
    #nodes_colors = cmap(norm(cases_ds["new_cases_by_100k"].loc[:, date]))
    plot_TE_on_graph2(DIxy, nodes, nodes_dict, topology, pos, ax=axs[0,i], hub_idx=hub, date=date,
                      colorbar=colorbar, cbar_type="nodes", nodes_color=nodes_colors)
    
    axs[0,i].set_title(f" ({labels[i]}) "+r"$DI_{XY,risk \rightarrow cases}$")
    axs[0,i].set_axis_off()
    axs[0,i].set_aspect("equal")
    
    nodes_colors = cmap(norm(DIx_cases_norm.loc[:, date]))
    plot_TE_on_graph2(DIxy_cases, nodes, nodes_dict, topology, pos, ax=axs[1,i], hub_idx=hub, date=date,
                      colorbar=colorbar, cbar_type="edges", nodes_color=nodes_colors)
   
    axs[1,i].set_title(f"({labels[i]}) "+r"$DI_{XY,cases \rightarrow cases}$")
    axs[1,i].set_axis_off()
    axs[1,i].set_aspect("equal")

axs[0,0].text(-0.1, 1.02, "C", fontsize=18, 
            transform=axs[0,0].transAxes, weight='bold', color='#333333')
axs[1,0].text(-0.1, 1.02, "D", fontsize=18, 
            transform=axs[1,0].transAxes, weight='bold', color='#333333')

for ax in axs.flatten():
    ax.set_anchor('NW')
    
fig.savefig("../figures/supp/FigureS3CD.tiff", dpi=300)

In [ ]:
R0 = "2"
topology = "ring"
hub = "M1"
target_folder = f"../outputs/SIR_simulation_{topology}/"

best_delta = 5#5 #bests[(R0,topology)][0]
best_omega = 9#9 #bests[(R0,topology)][1]

te_folder = os.path.join(target_folder, f"run_w{best_omega}_d{best_delta}_R_risk_ij/")
te_folder_cases = os.path.join(target_folder, f"run_w{best_omega}_d{best_delta}_R_new_cases_by_100k/")

Txy = xr.load_dataarray(os.path.join(te_folder, "TE.nc"))
DIxy = calculate_TSxy(Txy)

Txy_cases = xr.load_dataarray(os.path.join(te_folder_cases, "TE.nc"))
DIxy_cases = calculate_TSxy(Txy_cases)

params_fname = os.path.join(te_folder,"params.json")
with open(params_fname) as fh:
    params = json.load(fh)
omega = params["te_params"]["omega"]
delta = params["te_params"]["delta"]

cases_ds = xr.open_dataset(os.path.join(target_folder, "cases_pop_ds.nc"))
mobility_matrix = np.load(f"../data/SIR_mobility_matrices/mobility_{topology}_9.npy")

params_fname = os.path.join(target_folder, "config.json")
with open(params_fname) as fh:
    params = json.load(fh)
r0 = params["model_params"]["r0"]
mu = params["model_params"]["mu"]

fig,axs = plt.subplots(1, 2, figsize=(12,4) ,dpi=300)
fig.set_facecolor('white')
for ax in axs.flatten():
    ax.set_anchor('W')

plot_pop_on_graph(mobility_matrix, nodes, nodes_dict, topology, axs[0], hub)
axs[0].set_title("Mobility network")
axs[0].set_aspect(0.9)
axs[0].text(-0.1, 1.02, "A", fontsize=18, 
            transform=axs[0].transAxes, weight='bold', color='#333333')

n_days = 90
interval = 30
date_beg, date_end = plot_incidence(cases_ds, interval, n_days, axs[1], nodes_dict)
axs[1].set_title("Dynamics of the epidemic process ($R_0={:.1f}$, $\mu={:.2f}$)".format(r0,mu))
axs[1].set_aspect(4)
axs[1].text(-0.1, 1.02, "B", fontsize=18, 
            transform=axs[1].transAxes, weight='bold', color='#333333')

for ax in axs.flatten():
    ax.set_anchor('NW')
    
fig.savefig("../figures/supp/FigureS4AB.tiff", dpi=300)

#-----------------------------------------------------------------------------------------------------------------#

patches = nodes
mov = mobility_matrix 
edges_list = []
for i,pi in enumerate(patches):
    for j,pj in enumerate(patches):
        if mov[i,j] != 0 and i != j:
            edges_list.append((nodes_dict[pi],nodes_dict[pj], mov[i,j]))

G = nx.DiGraph()
G.add_nodes_from([nodes_dict[x] for x in patches])

for item in edges_list:
    G.add_edge(item[0],item[1],weight=int(item[2]))

if topology == "star":
    center_node="M1"
    edge_nodes = set(G) - {center_node}
    pos = nx.circular_layout(G.subgraph(edge_nodes))
    pos[center_node] = np.array([0, 0])
else:
    pos = nx.circular_layout(G)
    
node_sizes = 500

DIx_norm = calculate_normalized_TSx(Txy)
DIx_cases_norm = calculate_normalized_TSx(Txy_cases)

domain_min = -1.0
domain_max = 1.0
#domain_min = 0.0
#domain_max = cases_ds["new_cases_by_100k"].max()

norm = mcolors.Normalize(vmin=domain_min, vmax=domain_max)
cmap = sns.color_palette("vlag", as_cmap=True)
#cmap = plt.cm.Purples

all_dates = np.array([np.datetime_as_string(x)[:10] for x in list(DIxy["date"].values)])

days = [12, 15, 18, 21]
days = [11, 16, 26, 36]
dates = all_dates[days]


fig,axs = plt.subplots(2, 4, figsize=(12,6) ,dpi=400, sharex=True, sharey=True,
                       gridspec_kw={"width_ratios":[1,1,1,1.3], "height_ratios":[1,1], "wspace":0.1, "hspace":0.1})


fig.set_facecolor('white')
for ax in axs.flatten():
    ax.set_anchor('W')

    
labels = [f"D{x+delta}" for x in days]
for i, date in enumerate(dates):
    
    if i == 3:
        colorbar = True
    else:
        colorbar = False
    
    nodes_colors = cmap(norm(DIx_norm.loc[:, date]))
    #nodes_colors = cmap(norm(cases_ds["new_cases_by_100k"].loc[:, date]))
    plot_TE_on_graph2(DIxy, nodes, nodes_dict, topology, pos, ax=axs[0,i], hub_idx=hub, date=date,
                      colorbar=colorbar, cbar_type="nodes", nodes_color=nodes_colors)
    
    axs[0,i].set_title(f" ({labels[i]}) "+r"$DI_{XY,risk \rightarrow cases}$")
    axs[0,i].set_axis_off()
    axs[0,i].set_aspect("equal")
    
    nodes_colors = cmap(norm(DIx_cases_norm.loc[:, date]))
    plot_TE_on_graph2(DIxy_cases, nodes, nodes_dict, topology, pos, ax=axs[1,i], hub_idx=hub, date=date,
                      colorbar=colorbar, cbar_type="edges", nodes_color=nodes_colors)
   
    axs[1,i].set_title(f"({labels[i]}) "+r"$DI_{XY,cases \rightarrow cases}$")
    axs[1,i].set_axis_off()
    axs[1,i].set_aspect("equal")

axs[0,0].text(-0.1, 1.02, "C", fontsize=18, 
            transform=axs[0,0].transAxes, weight='bold', color='#333333')
axs[1,0].text(-0.1, 1.02, "D", fontsize=18, 
            transform=axs[1,0].transAxes, weight='bold', color='#333333')

for ax in axs.flatten():
    ax.set_anchor('NW')
    
fig.savefig("../figures/supp/FigureS4CD.tiff", dpi=300)